In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize

#設定資產
tickers = ['SHY', 'DBC', 'SPY']
data = yf.download(tickers, start='2006-03-01', end='2025-05-01')['Close']

monthly_prices = data.resample('M').last()
monthly_returns = monthly_prices.pct_change().dropna()

#定義我們要使用的資產
asset_columns = tickers

def portfolio_stats(weights, mean_returns, cov_matrix, risk_free_rate=0.00/12):
    returns = np.sum(mean_returns * weights)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = (returns - risk_free_rate) / volatility
    return returns, volatility, sharpe

def neg_sharpe(weights, mean_returns, cov_matrix, risk_free_rate=0.00/12):
    returns, volatility, sharpe = portfolio_stats(weights, mean_returns, cov_matrix, risk_free_rate)
    return -sharpe

def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_stats(weights, mean_returns, cov_matrix)[1]

def neg_returns(weights, mean_returns, cov_matrix):
    return -portfolio_stats(weights, mean_returns, cov_matrix)[0]


risk_free_rate = 0.00/12


window = 12


rolling_weights = []
rolling_dates = []


num_assets = len(asset_columns)


initial_weights = np.array([1/num_assets] * num_assets)

#限制式
bounds = tuple((0, 1) for _ in range(num_assets))
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# 滾動窗口
for end in range(window, len(monthly_returns)):
    start = end - window

    window_returns = monthly_returns.iloc[start:end]
    # 12個月平均報酬
    mean_returns = window_returns.mean()
    # 12個月共變異數
    cov_matrix = window_returns.cov()

    try:
        # Sharpe Ratio最佳化
        result = minimize(neg_sharpe,                    # 目標函數
                          initial_weights,               # 初始權重
                          args=(mean_returns, cov_matrix, risk_free_rate),  # 資料
                          method='SLSQP',                # 演算法
                          bounds=bounds,                 # 各資產權重限制
                          constraints=constraints)       # 總資產約束條件

        weights = result.x
        rolling_weights.append(weights)
        rolling_dates.append(monthly_returns.index[end])
    except Exception as e:
        print(f"優化失敗，日期: {monthly_returns.index[end]}, 錯誤: {str(e)}")


rolling_weights_df = pd.DataFrame(rolling_weights,
                                  index=rolling_dates,
                                  columns=asset_columns)


rolling_weights_df.round(6).to_csv('rolling_weights.csv')
print(rolling_weights_df)


first_optim_date = rolling_dates[0]
first_optim_idx = monthly_returns.index.get_loc(first_optim_date)

portfolio_returns = pd.Series(index=monthly_returns.index[first_optim_idx+1:])

for i in range(len(rolling_dates)):
    # 獲取當前日期和權重
    current_date = rolling_dates[i]
    current_weights = rolling_weights[i]

    # 找出下一個月的日期
    current_idx = monthly_returns.index.get_loc(current_date)
    if current_idx + 1 < len(monthly_returns):
        next_date = monthly_returns.index[current_idx + 1]
        # 獲取下一個月的報酬率
        next_returns = monthly_returns.loc[next_date]
        # 計算投資組合報酬率
        portfolio_returns[next_date] = np.sum(current_weights * next_returns)

portfolio_returns = portfolio_returns.dropna()

# 計算累積報酬率
cumulative_returns = (1 + portfolio_returns).cumprod() - 1

# 計算年化報酬率、波動率和夏普比率
annualized_return = (1 + portfolio_returns.mean()) ** 12 - 1
annualized_volatility = portfolio_returns.std() * np.sqrt(12)
sharpe_ratio = annualized_return / annualized_volatility if annualized_volatility > 0 else 0

print("\n============ 投資組合績效 ============")
print(f"投資組合年化報酬率: {annualized_return:.4f} ({annualized_return*100:.2f}%)")
print(f"投資組合年化波動率: {annualized_volatility:.4f} ({annualized_volatility*100:.2f}%)")
print(f"投資組合夏普值: {sharpe_ratio:.4f}")
print(f"投資組合總累積報酬率: {cumulative_returns.iloc[-1]:.4f} ({cumulative_returns.iloc[-1]*100:.2f}%)")

annual_returns = portfolio_returns.groupby(portfolio_returns.index.year).apply(lambda x: (1 + x).prod() - 1)
print("\n============ 每年報酬率 ============")
for year, ret in annual_returns.items():
    print(f"{year}年: {ret:.4f} ({ret*100:.2f}%)")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  3 of 3 completed
<ipython-input-1-4be18ff6d8a8>:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_prices = data.resample('M').last()


                     SHY       DBC       SPY
2007-04-30  0.000000e+00  0.969990  0.030010
2007-05-31  8.054851e-15  0.965938  0.034062
2007-06-30  0.000000e+00  0.884723  0.115277
2007-07-31  3.019105e-17  0.903392  0.096608
2007-08-31  2.168404e-19  0.910760  0.089240
...                  ...       ...       ...
2024-12-31  1.588473e-01  0.629902  0.211251
2025-01-31  1.842171e-01  0.673894  0.141889
2025-02-28  1.781579e-01  0.676321  0.145521
2025-03-31  1.515881e-01  0.827077  0.021335
2025-04-30  1.842770e-01  0.790284  0.025439

[217 rows x 3 columns]

============ 投資組合績效 ============
投資組合年化報酬率: 0.0230 (2.30%)
投資組合年化波動率: 0.0570 (5.70%)
投資組合夏普值: 0.4028
投資組合總累積報酬率: 0.4612 (46.12%)

============ 每年報酬率 ============
2007年: 0.0493 (4.93%)
2008年: -0.0088 (-0.88%)
2009年: 0.0040 (0.40%)
2010年: 0.0493 (4.93%)
2011年: 0.0117 (1.17%)
2012年: 0.0047 (0.47%)
2013年: 0.1237 (12.37%)
2014年: 0.0345 (3.45%)
2015年: 0.0075 (0.75%)
2016年: 0.0127 (1.27%)
2017年: 0.0355 (3.55%)
2018年: -0.0799 (-7.99%)
2019